In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

class Sentiment(BaseModel):
    """Define o sentimento e o idioma da mensagem enviada"""
    sentimento: str = Field(description="Sentimento do Texto. Deve ser 'pos', 'neg' ou 'nt' para não definido")
    lingua: str = Field(description="Língua que o texto foi escrito (deve estar no charset UTF-8)")
    
tool_sentimento = convert_to_openai_function(Sentiment)
tool_sentimento

/Users/thalesmarega/Documents/thalesmarega/workspace-python/estudos-agents/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'name': 'Sentiment',
 'description': 'Define o sentimento e o idioma da mensagem enviada',
 'parameters': {'type': 'object',
  'properties': {'sentimento': {'description': "Sentimento do Texto. Deve ser 'pos', 'neg' ou 'nt' para não definido",
    'type': 'string'},
   'lingua': {'description': 'Língua que o texto foi escrito (deve estar no charset UTF-8)',
    'type': 'string'}},
  'required': ['sentimento', 'lingua']}}

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Analise o texto e categorize-o conforme as instruções"),
    ("user", "{input}")
])

chat = ChatOpenAI()

texto = "Eu gosto muito de pizza nordestina"

chain = prompt | chat.bind(functions=[tool_sentimento], 
                           function_call={"name": "Sentiment"})

chain.invoke({"input": texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"pos","lingua":"portuguese"}', 'name': 'Sentiment'}}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 137, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9bfa7700-b6a8-4d58-a765-15487b51b32c-0')

In [5]:
chain.invoke({"input": "May the Force be with you"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"nt","lingua":"en"}', 'name': 'Sentiment'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 134, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9784e457-4466-4a85-a892-6ae4fd3b234e-0')

### Categorização de Texto

In [6]:
solicitacoes = [
    "Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?",
    "Não consigo acessar minha conta. A senha não está funcionando!",
    "Estou tendo dificuldades para instalar o software de design gráfico. Pode me ajudar?",
    "Gostaria de saber como atualizar meu endereço de entrega na plataforma.",
    "O meu laptop não está ligando. Acho que é um problema no cabo de energia.",
    "Não consigo ver meus arquivos no aplicativo. Existe alguma solução?"
]

from enum import Enum
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

class SetorEnum(str, Enum):
    suporte_hardware = "suporte_hardware"
    suporte_software = "suporte_software"
    suporte_conta = "suporte_conta"
    outros = "outros"
    
class DirecionaSuporte(BaseModel):
    """Direciona a solicitação de suporte para o setor responsável"""
    setor: SetorEnum

tool_direcionamento_suporte = convert_to_openai_function(DirecionaSuporte)
tool_direcionamento_suporte

{'name': 'DirecionaSuporte',
 'description': 'Direciona a solicitação de suporte para o setor responsável',
 'parameters': {'type': 'object',
  'properties': {'setor': {'description': 'An enumeration.',
    'enum': ['suporte_hardware',
     'suporte_software',
     'suporte_conta',
     'outros'],
    'type': 'string'}},
  'required': ['setor']}}

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Criar o prompt com as instruções detalhadas para o modelo
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a problemas de hardware, como falha no computador, laptop ou 
outros dispositivos físicos devem ser direcionadas para "suporte_hardware".
Questões relacionadas a problemas com software, como instalação, erros ao abrir programas, 
etc., devem ser direcionadas para "suporte_software".
Problemas relacionados ao acesso de conta, como recuperação de senha, problemas de login, 
devem ser direcionados para "suporte_conta".
Mensagens que não se encaixam nessas categorias devem ser direcionadas para "outros".
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_direcionamento_suporte],
                            function_call={"name":"DirecionaSuporte"})
         | JsonOutputFunctionsParser())

solicitacao = solicitacoes[0]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?
Resposta: {'setor': 'suporte_software'}


In [8]:
solicitacao = solicitacoes[3]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: Gostaria de saber como atualizar meu endereço de entrega na plataforma.
Resposta: {'setor': 'suporte_conta'}


In [9]:
solicitacao = solicitacoes[4]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: O meu laptop não está ligando. Acho que é um problema no cabo de energia.
Resposta: {'setor': 'suporte_hardware'}
